Imports necessários:

In [1]:
import pandas as pd

Lendo o arquivo e armazenando em uma lista:

In [2]:
filepath = 'C:\\Users\\User\\Desktop\\8º período\\DM\\TP1\\itemset_mining_applied_to_pokemon_teams\\'

txt_file = open(filepath + 'data\\original\\gen8ou.txt', 'r', encoding="utf8")
content_as_list = txt_file.readlines()
txt_file.close()

Pegando as posições dos delimitadores de equipes no arquivo (cabeçalhos que começam com '==='):

In [3]:
content_as_dict = dict(enumerate(content_as_list))

In [4]:
delimiter_positions = []

for key, value in content_as_dict.items():
    if value.startswith('==='):
        delimiter_positions.append(key)

Criando um dicionário em que a chave é a posição de um delimitador e o valor é a posição do próximo delimitador:

In [5]:
shifted_delimiter_positions = delimiter_positions[1:]
shifted_delimiter_positions.append(277010)
delimiter_positions_dic = dict(zip(delimiter_positions, shifted_delimiter_positions))

Dividindo a lista original em uma lista de blocos de texto que representam as equipes:

In [6]:
team_list = []

for begin, end in delimiter_positions_dic.items():
    team_list.append(content_as_list[begin:end])

Transformando a lista de equipes em uma lista de dicionários em que cada linha do bloco se torna um valor, a fim de facilitar a manipulação posteriormente:

In [7]:
team_as_list_of_dict = []

for i in team_list:
    team_as_list_of_dict.append(dict(enumerate(i)))

Gerando uma lista em que cada elemento é uma lista contendo os nomes dos pokémons da equipe, deixando de lado as informações que não serão utilizadas:

In [8]:
formatted_team_list = []

for block in team_as_list_of_dict:
    team = []
    for index, line in block.items():
        if line.startswith('Ability'): # A linha acima da que começa com 'Ability' na maioria das vezes contém o nome
            if block[index-1].startswith('Shiny:'): # Em alguns casos contém a especificação de pokémon shiny
                team.append(block[index-2].split(' @')[0]) # Deixando de lado as informações sobre item utilizado
            else:
                team.append(block[index-1].split(' @')[0]) # Deixando de lado as informações sobre item utilizado
    formatted_team_list.append(team)

Pegando apenas equipes de 6 pokémons:

In [9]:
teams_with_six_members = []

for team in formatted_team_list:
    if len(team) == 6:
        teams_with_six_members.append(team)
        
print(f'Existem {len(teams_with_six_members)} equipes de 6 pokémons na base!' )

Existem 4342 equipes de 6 pokémons na base!


Agora vamos remover as informações de sexo dos pokémons, pois essa diferenciação irá apenas nos atrapalhar:

In [10]:
teams_without_male_sex_information = []

for team in teams_with_six_members:
    pokemons = []
    for pokemon in team:
        pokemons.append(pokemon.split(' (M)'))
    teams_without_male_sex_information.append(pokemons)

teams_without_sex_information = []

for team in teams_without_male_sex_information:
    pokemons = []
    for pokemon_as_list in team:
        for element in pokemon_as_list:
            pokemons.append(element.split(' (F)'))
    teams_without_sex_information.append(pokemons)

pokemon_teams = []

for team in teams_without_sex_information:
    pokemons = []
    for pokemon_as_list in team:
        for element in pokemon_as_list:
            if element != [''] and element != '' and element != '  \n' and element != ' \n':
                pokemons.append(element)
    pokemon_teams.append(pokemons)

Separando em dois os nomes dos pokémons que possuem apelidos atribuídos:

In [11]:
teams_with_separate_nicknames = []

for team in pokemon_teams:
    pokemons = []
    for pokemon in team:
        pokemons.append(pokemon.split(' ('))
    teams_with_separate_nicknames.append(pokemons)

Montando um dataframe com as equipes:

In [12]:
columns = ['pokemon_1', 'pokemon_2', 'pokemon_3', 'pokemon_4', 'pokemon_5', 'pokemon_6']
team_df = pd.DataFrame(columns=columns)

for index, team in enumerate(teams_with_separate_nicknames):
    team_df.loc[index] = team
    
team_df.head(15)

,pokemon_1,pokemon_2,pokemon_3,pokemon_4,pokemon_5,pokemon_6
0,[Landorus-Therian],[Corviknight],[Tapu Fini],[Slowking-Galar],[Volcarona],[Dragapult]
1,[Hawlucha],[Ninetales-Alola],[Garchomp],[Cloyster],[Tapu Lele],[Heatran]
2,[Kartana],[Zapdos],[Heatran],[Landorus-Therian],[Dragapult],[Toxapex]
3,[Heatran],[Melmetal],[Tornadus-Therian],[Landorus-Therian],[Slowbro],[Weavile]
4,"[Lumberjack, Diggersby)]","[Wyatt, Shuckle)]","[Leviathan, Dragonite)]","[Shellshock, Blastoise)]","[Quintessa, Tapu Koko)]","[Achilles, Aegislash)]"
5,[Tapu Koko],[Landorus-Therian],[Volcanion],[Kartana],[Weavile],[Buzzwole]
6,[Gyarados],[Heatran],[Garchomp],[Mew],[Zapdos],[Weavile]
7,[Tapu Koko],[Hippowdon],[Tornadus-Therian],[Skarmory],[Reuniclus],[Toxapex]
8,[Silvally-Electric],[Zeraora],[Landorus-Therian],[Tapu Fini],[Tornadus-Therian],[Heatran]
9,[Landorus-Therian],[Corviknight],[Toxapex],[Blacephalon],[Heatran],[Dragapult]


Removendo os apelidos:

In [13]:
columns = team_df.columns.to_list()

for column in columns:
    team_df[column] = [pokemon[1][:-1] if len(pokemon) == 2 else str(pokemon)[2:-2] for pokemon in team_df[column]]
    
team_df.head(15)

,pokemon_1,pokemon_2,pokemon_3,pokemon_4,pokemon_5,pokemon_6
0,Landorus-Therian,Corviknight,Tapu Fini,Slowking-Galar,Volcarona,Dragapult
1,Hawlucha,Ninetales-Alola,Garchomp,Cloyster,Tapu Lele,Heatran
2,Kartana,Zapdos,Heatran,Landorus-Therian,Dragapult,Toxapex
3,Heatran,Melmetal,Tornadus-Therian,Landorus-Therian,Slowbro,Weavile
4,Diggersby,Shuckle,Dragonite,Blastoise,Tapu Koko,Aegislash
5,Tapu Koko,Landorus-Therian,Volcanion,Kartana,Weavile,Buzzwole
6,Gyarados,Heatran,Garchomp,Mew,Zapdos,Weavile
7,Tapu Koko,Hippowdon,Tornadus-Therian,Skarmory,Reuniclus,Toxapex
8,Silvally-Electric,Zeraora,Landorus-Therian,Tapu Fini,Tornadus-Therian,Heatran
9,Landorus-Therian,Corviknight,Toxapex,Blacephalon,Heatran,Dragapult


Retirando os espaços do início e fim:

In [14]:
for column in columns:
    team_df[column] = [pokemon.strip() for pokemon in team_df[column]]

Limpando nomes que permaneceram com ' \n' no final:

In [15]:
for column in columns:
    team_df[column] = [pokemon[:-3] if pokemon.endswith('  \\n') else pokemon for pokemon in team_df[column]]
    team_df[column] = [pokemon[:-2] if pokemon.endswith('\\n') else pokemon for pokemon in team_df[column]]
team_df.head(15)

,pokemon_1,pokemon_2,pokemon_3,pokemon_4,pokemon_5,pokemon_6
0,Landorus-Therian,Corviknight,Tapu Fini,Slowking-Galar,Volcarona,Dragapult
1,Hawlucha,Ninetales-Alola,Garchomp,Cloyster,Tapu Lele,Heatran
2,Kartana,Zapdos,Heatran,Landorus-Therian,Dragapult,Toxapex
3,Heatran,Melmetal,Tornadus-Therian,Landorus-Therian,Slowbro,Weavile
4,Diggersby,Shuckle,Dragonite,Blastoise,Tapu Koko,Aegislash
5,Tapu Koko,Landorus-Therian,Volcanion,Kartana,Weavile,Buzzwole
6,Gyarados,Heatran,Garchomp,Mew,Zapdos,Weavile
7,Tapu Koko,Hippowdon,Tornadus-Therian,Skarmory,Reuniclus,Toxapex
8,Silvally-Electric,Zeraora,Landorus-Therian,Tapu Fini,Tornadus-Therian,Heatran
9,Landorus-Therian,Corviknight,Toxapex,Blacephalon,Heatran,Dragapult


Vamos dar uma conferida nos pokémons que existem no dataset e analisar se restou algum lixo ou algum pokémon com nome errado:

In [16]:
pokemons_existentes = team_df.stack().reset_index(drop=True)
pokemons_existentes = pd.Series([pokemon for pokemon in pokemons_existentes])
for i in list(pokemons_existentes.unique()):
    print(i)

Landorus-Therian
Corviknight
Tapu Fini
Slowking-Galar
Volcarona
Dragapult
Hawlucha
Ninetales-Alola
Garchomp
Cloyster
Tapu Lele
Heatran
Kartana
Zapdos
Toxapex
Melmetal
Tornadus-Therian
Slowbro
Weavile
Diggersby
Shuckle
Dragonite
Blastoise
Tapu Koko
Aegislash
Volcanion
Buzzwole
Gyarados
Mew
Hippowdon
Skarmory
Reuniclus
Silvally-Electric
Zeraora
Blacephalon
Torkoal
Venusaur
Xatu
Altaria
Toxapex 
Shedinja 
Magnezone 
Corviknight 
Blissey
Xurkitree
Shedinja
Pyukumuku
Magnezone
Primarina
Ferrothorn
Mareanie
Excadrill
Pelipper
Barraskewda
Moltres-Galar
Seismitoad
Weezing-Galar
Clefable
Suicune
Gastrodon-East
Snorlax
Polteageist
Crawdaunt
Quagsire
Rotom-Wash
Zygarde-10%
Urshifu-Rapid-Strike
Scizor
Tyrantrum
Bisharp
Rillaboom
Thundurus-Therian
Scolipede
Thundurus
Victini
Tangrowth
Nihilego
Celesteela
Jirachi
Slowking
Tyranitar
Hydreigon
Haxorus
Cobalion
Kommo-o
Togekiss
Gastrodon
Omastar
Gengar
Chansey
Raichu-Alola
Dracozolt
Zapdos-Galar
Arctozolt
Latias
Nidoking
Blaziken
Latios
Regieleki
Clefa

Descobrindo as linhas que contém erro de codificação e corrigindo:

In [17]:
for column in columns:
    print(team_df[team_df[column] == 'Ri &#34;BOMB&#34; ee'].index.to_list())
    print(team_df[team_df[column] == 'Farfetch&#39;d'].index.to_list())
    print(team_df[team_df[column] == 'Don&#39;t'].index.to_list())
    print(team_df[team_df[column] == 'Sirfetch&#39;d'].index.to_list())


[3468]
[]
[]
[3800, 3904]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[2462]
[]
[256]
[]
[]
[]
[]
[]
[]
[716]
[]


In [18]:
positions_with_error = [256, 716, 2462, 3468, 3800, 3904]
for position in positions_with_error:
    print(team_df.loc[position])

pokemon_1         Hatterene
pokemon_2          Vaporeon
pokemon_3           Snorlax
pokemon_4    Sirfetch&#39;d
pokemon_5         Aegislash
pokemon_6           Zeraora
Name: 256, dtype: object
pokemon_1    Landorus-Therian
pokemon_2           Tapu Lele
pokemon_3            Alakazam
pokemon_4          Ferrothorn
pokemon_5       Moltres-Galar
pokemon_6           Don&#39;t
Name: 716, dtype: object
pokemon_1          Smeargle
pokemon_2          Greninja
pokemon_3         Stakataka
pokemon_4    Farfetch&#39;d
pokemon_5           Toxapex
pokemon_6           Marowak
Name: 2462, dtype: object
pokemon_1    Ri &#34;BOMB&#34; ee
pokemon_2                 Mimikyu
pokemon_3               Excadrill
pokemon_4               Dracovish
pokemon_5              Toxtricity
pokemon_6                 Bisharp
Name: 3468, dtype: object
pokemon_1     Sirfetch&#39;d
pokemon_2    Grimmsnarl-Gmax
pokemon_3          Aegislash
pokemon_4        Corviknight
pokemon_5         Seismitoad
pokemon_6              Ditto
Name

In [19]:
team_df.loc[256]['pokemon_4'] = 'Sirfetch’d'
print(team_df.loc[256]['pokemon_4'])
team_df.loc[716]['pokemon_6'] = 'Toxapex'
print(team_df.loc[716]['pokemon_6'])
team_df.loc[2462]['pokemon_4'] = 'Farfetch’d'
print(team_df.loc[2462]['pokemon_4'])
team_df.loc[3468]['pokemon_1'] = 'Ribombee'
print(team_df.loc[3468]['pokemon_1'])
team_df.loc[3800]['pokemon_1'] = 'Sirfetch’d'
print(team_df.loc[3800]['pokemon_1'])
team_df.loc[3904]['pokemon_1'] = 'Sirfetch’d'
print(team_df.loc[3904]['pokemon_1'])

Sirfetch’d
Toxapex
Farfetch’d
Ribombee
Sirfetch’d
Sirfetch’d


Algumas linhas permaneceram com lixo devido a alguns trechos da base original que fugiam do padrão, abaixo estamos limpando essas linhas:

In [20]:
trash_str1 = "', '"
trash_str2 = ">"

for column in columns:
    team_df[column] = [pokemon.split("', '")[-1][:-1] if trash_str1 in pokemon else pokemon for pokemon in team_df[column]]
    team_df[column] = [pokemon.split(">")[-1] if trash_str2 in pokemon else pokemon for pokemon in team_df[column]]
    team_df[column] = [pokemon[:-1] if pokemon.endswith(')') else pokemon for pokemon in team_df[column]]

starting_with_evs_filter = team_df['pokemon_1'].str.startswith('EVs:')

team_df.drop(team_df[starting_with_evs_filter].index, inplace=True)


    
team_df

,pokemon_1,pokemon_2,pokemon_3,pokemon_4,pokemon_5,pokemon_6
0,Landorus-Therian,Corviknight,Tapu Fini,Slowking-Galar,Volcarona,Dragapult
1,Hawlucha,Ninetales-Alola,Garchomp,Cloyster,Tapu Lele,Heatran
2,Kartana,Zapdos,Heatran,Landorus-Therian,Dragapult,Toxapex
3,Heatran,Melmetal,Tornadus-Therian,Landorus-Therian,Slowbro,Weavile
4,Diggersby,Shuckle,Dragonite,Blastoise,Tapu Koko,Aegislash
...,...,...,...,...,...,...
4337,Sylveon,Flygon,Inteleon,Arcanine,Toxtricity,Vileplume
4338,Hydreigon,Clefable,Corviknight,Seismitoad,Obstagoon,Centiskorch
4339,Ferrothorn,Toxapex,Rotom-Heat,Clefable,Excadrill,Dragapult
4340,Toxapex,Mew,Clefable,Seismitoad,Tyranitar,Excadrill


Finalmente, base de dados tratada \o/

Vamos exportá-la como .csv e .xlsx:

In [21]:
team_df.to_csv(filepath + 'data\\processed\\pokemon_teams.csv', index=False)
team_df.to_excel(filepath + 'data\\processed\\pokemon_teams.xlsx', index=False)

In [22]:
pokemons_existentes = team_df.stack().reset_index(drop=True)
pokemons_existentes = pd.Series([pokemon for pokemon in pokemons_existentes])